<a target="_blank" href="https://colab.research.google.com/github/rajshah4/LLM-Evaluation/blob/main/ragas_quickstart.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Welcome to the ragas quickstart. We're going to get you up and running with ragas as qickly as you can so that you can go back to improving your Retrieval Augmented Generation pipelines while this library makes sure your changes are improving your entire pipeline.

to kick things of lets start with the data

In [28]:
%pip -q install ragas==0.1.13 pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.8 MB/s eta 0:00:00


Ragas also uses OpenAI for running some metrics so make sure you have your openai key ready and available in your environment

In [1]:
import os
import getpass

open_ai_key = getpass.getpass('Enter your OPENAI API Key')
os.environ['OPENAI_API_KEY'] = open_ai_key

Enter your OPENAI API Key··········


In [2]:
from datasets import load_dataset, Dataset

hindi_dataset = load_dataset("explodinggradients/amnesty_qa","hindi_v1")
hindi_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [3]:

(dict(hindi_dataset)['eval']).to_pandas()

,question,ground_truths,answer,contexts
0,यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वैश...,[यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वै...,यूएसए सुप्रीम कोर्ट के गर्भपात पर फैसले के वैश...,"[- 2022 में, अमेरिका के सर्वोच्च न्यायालय ने ए..."
1,"कार्बन मेजर्स डेटाबेस के अनुसार, GHG उत्सर्जन ...","[कार्बन मेजर्स डेटाबेस के अनुसार, GHG उत्सर्जन...","fuel industry, are responsible for a significa...","[- फॉसिल ईंधन कंपनियां, चाहे वे राज्य स्वामित्..."
2,कार्बन मेजर्स डेटाबेस के अनुसार अमेरिकास में क...,"[कार्बन मेजर्स डेटाबेस के अनुसार, अमेरिका में ...","कार्बन मेजर्स डेटाबेस के अनुसार, अमेरिकास में ...","[डेटाबेस के अनुसार, इस अवधि के दौरान सबसे अधिक..."
3,अमनेस्टी इंटरनेशनल ने ओगोनी 9 की हत्या के जवाब...,[अमनेस्टी इंटरनेशनल ने अपने समर्थकों से अपील क...,अमनेस्टी इंटरनेशनल ने अपने समर्थकों से अनुरोध ...,[अमनेस्टी इंटरनेशनल ने अपने विशाल समर्थक नेटवर...
4,एमनेस्टी इंटरनेशनल ने मानवाधिकार रक्षकों पर वि...,[एमनेस्टी इंटरनेशनल द्वारा मानवाधिकार रक्षकों ...,अमनेस्टी इंटरनेशनल ने मानवाधिकार रक्षकों पर वि...,[एमनेस्टी इंटरनेशनल की सिफारिश है कि मानवाधिका...
5,एमनेस्टी इंटरनेशनल द्वारा बच्चों के अधिकारों प...,[एमनेस्टी इंटरनेशनल द्वारा बाल अधिकारों पर बना...,एमनेस्टी इंटरनेशनल द्वारा बाल अधिकारों पर बनाई...,[इसलिए अमनेस्टी इंटरनेशनल ने बच्चों के अधिकारो...
6,कौन सा अधिकार व्यापक जानकारी की गारंटी देता है...,[वह अधिकार जो पिछले मानवाधिकार उल्लंघनों के बा...,वह अधिकार जो पिछले मानवाधिकार उल्लंघनों के बार...,[26. यह अधिनियम सत्य के अधिकार के साथ अपनी संग...
7,"मानवाधिकार उल्लंघनों के बारे में पूरी जानकारी,...",[मानवाधिकारों की घोर उल्लंघन के शिकार और उनके ...,हर किसी का अधिकार होता है कि उसे मानवाधिकार उल...,[- मानवाधिकारों की घोर उल्लंघन के शिकार और उनक...
8,रूसी आपराधिक संहिता की धारा 207.3 के तहत व्यक्...,[यदि किसी व्यक्ति के बयान रूसी अधिकारियों के आ...,"रूसी आपराधिक संहिता की अनुच्छेद 207.3 के तहत, ...",[- जब तक उनके बयान रूसी अधिकारियों के आधिकारिक...
9,अनुच्छेद 207.3 के तहत अभियोग न्यायालय कब आधिका...,[अभियोग पक्ष आधिकारिक स्थिति के विपरीत बयानों ...,"रूसी आपराधिक संहिता की अनुच्छेद 207.3 के तहत, ...",[- जब तक उनके बयान रूसी अधिकारियों के आधिकारिक...


In [8]:
from ragas.metrics import (
    faithfulness,
    answer_correctness,
)
from langchain.chat_models import ChatOpenAI
from ragas import adapt

# llm used for adaptation
openai_model = ChatOpenAI(model_name="gpt-4")

adapt(metrics=[faithfulness,answer_correctness], language="spanish", llm=openai_model)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

## The Data

Ragas performs a `ground_truth` free evaluation of your RAG pipelines. This is because for most people building a gold labeled dataset which represents in the distribution they get in production is a very expensive process.

Hence to work with ragas all you need are the following data
- question: `list[str]` - These are the questions you RAG pipeline will be evaluated on.
- answer: `list[str]` - The answer generated from the RAG pipeline and give to the user.
- contexts: `list[list[str]]` - The contexts which where passed into the LLM to answer the question.

Ideally your list of questions should reflect the questions your users give, including those that you have been problamatic in the past.

Here we're using an example dataset from on of the baselines we created for the [Financial Opinion Mining and Question Answering (fiqa) Dataset](https://sites.google.com/view/fiqa/) we created. If you want to want to know more about the baseline, feel free to check the `experiements/baseline` section

In [24]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

README.md:   0%|          | 0.00/112 [00:00<?, ?B/s]

fiqa.py:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/115k [00:00<?, ?B/s]

Generating baseline split:   0%|          | 0/30 [00:00<?, ? examples/s]

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

## Metrics

Ragas measures your pipeline's performance against two dimensions

1. Faithfulness: measures the factual consistency of the generated answer against the given context.
2. Relevancy: measures how relevant retrieved contexts and the generated answer are to the question.

Through repeated experiments, we have found that the quality of a RAG pipeline is highly dependent on these two dimensions. The final `ragas_score` is the harmonic mean of these two factors.

now lets import these metrics and understand more about what they denote

In [25]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision
)

here you can see that we are using 4 metrics, but what do the represent?

1. answer_relevancy - a measure of how relevant the answer is to the question

2. faithfulness - the factual consistency of the answer to the context base on the question.

3. context_recall: measures the ability of the retriever to retrieve all the necessary information needed to answer the question.

4. context_precision - a measure of how relevant the retrieved context is to the question. Conveys quality of the retrieval pipeline.

**Note:** *by default these metrics are using OpenAI's API to compute the score. If you using this metric make sure you set the environment key `OPENAI_API_KEY` with your API key. You can also try other LLMs for evaluation, check the [llm guide](./guides/llms.ipynb) to learn more*

If you're interested in learning more, feel free to check the [docs](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

## Evaluation

Running the evalutation is as simple as calling evaluate on the `Dataset` with the metrics of your choice.

In [26]:
from ragas import evaluate

result = evaluate(
    fiqa_eval["baseline"].select(range(1)),
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
    ],
)

result

ValueError: The metric [context_precision] that is used requires the following additional columns ['reference'] to be present in the dataset.

and there you have the it, all the scores you need. `ragas_score` gives you a single metric that you can use while the other onces measure the different parts of your pipeline.

now if we want to dig into the results and figure out examples where your pipeline performed worse or really good you can easily convert it into a pandas array and use your standard analytics tools too!

In [ ]:
df = result.to_pandas()
df.head()

,question,contexts,answer,ground_truths,context_precision,faithfulness,answer_relevancy,context_recall
0,How to deposit a cheque issued to an associate...,[Just have the associate sign the back and the...,\nThe best way to deposit a cheque issued to a...,[Have the check reissued to the proper payee.J...,0.470588,0.666667,0.976913,0.111111


And thats it!

You can check out the [ragas in action] notebook to get a feel of what is like to use it while trying to improve your pipelines.

if you have any suggestion/feedbacks/things your not happy about, please do share it in the [issue section](https://github.com/explodinggradients/ragas/issues). We love hearing from you 😁